# **Importación de Librerías**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
import tempfile
import torch
import torch.nn as nn
import torch.optim as optim

from torchinfo import summary
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import datasets
from torchvision import transforms
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Lambda, Compose

from tqdm.notebook import tqdm

from pathlib import Path

In [ ]:
import mlflow

HOST = "127.0.0.1"
PORT = 8080
TRACKING_SERVER_URI = f"http://{HOST}:{PORT}"

mlflow.set_tracking_uri(TRACKING_SERVER_URI)

In [ ]:
import mlflow.exceptions

EXPERIMENT_DESCRIPPTION = (
    "Convolutional Autoencoder with PyTorch and MNIST dataset"
)

EXPERIMENT_TAGS = {
    "project_name": "NeuralNetworks_FAMAF_2024",
    "architecture": "ConvolutionalAutoencoder",
    "author": "bbas",
    "mlflow.note.content": EXPERIMENT_DESCRIPPTION
}

try:
    mlflow.create_experiment(name="ConvAutoencoder-FashionMNIST", tags=EXPERIMENT_TAGS)
except mlflow.exceptions.RestException as e:
    print(e)
mlflow.set_experiment("ConvAutoencoder-FashionMNIST")

# **Carga de Datos**

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

train_set_orig = datasets.FashionMNIST('MNIST_data/', download=True, train=True , transform=transform)
valid_set_orig = datasets.FashionMNIST('MNIST_data/', download=True, train=False, transform=transform)

# **CustomDataset**

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        input = output = self.dataset[idx][0]
        return input, output

# **Funciones**

## Una época de entrenamiento

In [ ]:
def train_step(model, dataloader, loss_fn, optimizer):
    """
    Trains the model for ONE epoch = goes over the dataset one time using the batches.
    """
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    
    is_classification = isinstance(loss_fn, nn.CrossEntropyLoss)
    total_acc = 0 if is_classification else None
    total_loss = 0
    
    model_device = next(model.parameters()).device
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(model_device), y.to(model_device)

        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        if is_classification:
            total_acc += (y_pred.argmax(1) == y).float().sum().item()

        if batch % 100 == 0:
            print(f"Batch {batch} of {num_batches}. Loss in batch: {loss.item():.4f}")

    avg_loss = total_loss / num_batches
    acc = total_acc / size if is_classification else None

    return acc, avg_loss

## Validación post época

In [ ]:
def validate_step(model, dataloader, loss_fn):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    
    is_classification = isinstance(loss_fn, nn.CrossEntropyLoss)
    total_acc = 0 if is_classification else None
    total_loss = 0

    model_device = next(model.parameters()).device
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(model_device), y.to(model_device)

            y_pred = model(X)
            total_loss += loss_fn(y_pred, y).item()

            if is_classification:
                total_acc += (y_pred.argmax(1) == y).float().sum().item()

    avg_loss = total_loss / num_batches
    acc = total_acc / size if is_classification else None

    return acc, avg_loss

## Early Stopping basado en loss

In [ ]:
class EarlyStopping():
    def __init__(self, delta=0.0001, patience=10):
        self.best_loss = np.inf
        self.best_epoch = 0
        self.best_model = None
        self.delta = delta
        self.patience = patience
        self.counter = 0
    
    def update(self, model, epoch, loss):
        early_stop = False
        if loss < self.best_loss - self.delta:
            self.best_loss = loss
            self.best_epoch = epoch
            self.best_model = model.state_dict()
            self.counter = 0
        else:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter == self.patience:
                early_stop = True
                print(f"EarlyStopping: Stopped training at epoch {epoch}.")
        return early_stop

## Loop de entrenamiento - validación

In [ ]:
def log_hyperparams(hyperparams):
    mlflow.log_params(hyperparams)

def log_model_architecture(model):
    with tempfile.TemporaryDirectory() as tmp:
        path = Path(tmp, "model_architecture.txt")
        path.write_text(str(model))
        mlflow.log_artifact(path)

def train_validate_loop(
    model, train_loader, valid_loader, loss_fn, optimizer, epochs, extra_hyperparams={},
    early_stopper=None,
):
    is_classification = isinstance(loss_fn, nn.CrossEntropyLoss)
    
    hyperparams = {
        "max_epochs": epochs,
        "batch_size": train_loader.batch_size,
        "optimizer": optimizer.__class__.__name__,
        "learning_rate": optimizer.defaults['lr'],
        "loss_function": loss_fn.__class__.__name__,
        "early_stopper": {
            "enabled": early_stopper is not None,
            "patience": early_stopper.patience if early_stopper else None,
            "delta": early_stopper.delta if early_stopper else None
        },
        **extra_hyperparams
    }
    log_hyperparams(hyperparams)
    log_model_architecture(model)

    accs_training, train_accs, valid_accs = ([], [], []) if is_classification else None, None, None
    avg_losses_training, train_avg_losses, valid_avg_losses = ([], [], [])

    for epoch in tqdm(range(1, epochs+1)):
        tqdm.write(f"Epoch {epoch}")
        acc_training, avg_loss_training = train_step(model, train_loader, loss_fn, optimizer)

        train_acc, train_avg_loss = validate_step(model, train_loader, loss_fn)
        valid_acc, valid_avg_loss = validate_step(model, valid_loader, loss_fn)

        avg_losses_training.append(avg_loss_training)
        train_avg_losses.append(train_avg_loss)
        valid_avg_losses.append(valid_avg_loss)
        mlflow.log_metric("Average Loss during Training", f"{avg_loss_training:.6f}", step=epoch)
        mlflow.log_metric("Average Loss in Training Set", f"{train_avg_loss:.6f}", step=epoch)
        mlflow.log_metric("Average Loss in Validation Set", f"{valid_avg_loss:.6f}", step=epoch)
        tqdm.write(f"Train avg loss: {train_avg_loss:.6f} | Valid avg loss: {valid_avg_loss:.6f}")
        
        if is_classification:
            accs_training.append(acc_training)
            train_accs.append(train_acc)
            valid_accs.append(valid_acc)
            mlflow.log_metric("Accuracy during Training", f"{acc_training:.6f}", step=epoch)
            mlflow.log_metric("Accuracy in Training Set", f"{train_acc:.6f}", step=epoch)
            mlflow.log_metric("Accuracy in Validation Set", f"{valid_acc:.6f}", step=epoch)
            tqdm.write(f"Train accuracy: {train_acc:.6f} | Valid accuracy: {valid_acc:.6f}")
 
        if early_stopper and early_stopper.update(model, epoch, valid_avg_loss):
            break

        tqdm.write("----------------------------------------------------------------")

    mlflow.log_metric("Last Epoch", epoch)
    print(f"Training finished! Trained for {epoch} epochs.")
    print(
        f"Final results from epoch {epoch}:\n"
        f"  - Train avg loss: {train_avg_loss:.6f}\n"
        f"  - Valid avg loss: {valid_avg_loss:.6f}"
    )

    return {
        'model': model,
        'avg_losses_training': avg_losses_training,
        'train_avg_losses': train_avg_losses,
        'valid_avg_losses': valid_avg_losses,
        'accs_training': accs_training,
        'train_accs': train_accs,
        'valid_accs': valid_accs
    }

## Gráfico de accuracy y pérdidas

In [ ]:
def losses_plot(avg_losses_training, train_avg_losses, valid_avg_losses, show=False):
    epochs = len(train_avg_losses)
    epochs_range = range(epochs)
    
    plt.figure(figsize=(8, 6))
    plt.plot(epochs_range, avg_losses_training, label="Durante la época", linestyle="--", color="blue")
    plt.plot(epochs_range, train_avg_losses, label="Entrenamiento", linestyle="-", color="green")
    plt.plot(epochs_range, valid_avg_losses, label="Validación", linestyle=":", color="red")
    
    plt.title("Pérdidas durante el entrenamiento")
    plt.xlabel("Épocas")
    plt.ylabel("Pérdida promedio por lote")
    plt.legend()
    plt.grid()
    plt.tight_layout()

    with tempfile.TemporaryDirectory() as tmp:
        path = Path(tmp, "losses_plot.png")
        plt.savefig(path)
        mlflow.log_artifact(path)

    if show:
        plt.show()

def accs_plot(accs_training, train_accs, valid_accs, show=False):
    epochs = len(train_accs)
    epochs_range = range(epochs)
    
    plt.figure(figsize=(8, 6))
    plt.plot(epochs_range, accs_training, label="Durante la época", linestyle="--", color="blue")
    plt.plot(epochs_range, train_accs, label="Entrenamiento", linestyle="-", color="green")
    plt.plot(epochs_range, valid_accs, label="Validación", linestyle=":", color="red")
    
    plt.title("Accuracy durante el entrenamiento")
    plt.xlabel("Épocas")
    plt.ylabel("Accuracy (exactitud)")
    plt.legend()
    plt.grid()
    plt.tight_layout()

    with tempfile.TemporaryDirectory() as tmp:
        path = Path(tmp, "accuracy_plot.png")
        plt.savefig(path)
        mlflow.log_artifact(path)

    if show:
        plt.show()

## Imagen original y reconstruida

In [ ]:
def plot_orig_predicted(model, train_set, num_samples=3, show=False):
    model_device = next(model.parameters()).device
    model.eval()

    num_samples = 2
    fig, axes = plt.subplots(nrows=num_samples, ncols=2, figsize=(8, 2*num_samples))

    for i in range(num_samples):
        sample_idx = torch.randint(len(train_set), size=(1,)).item()
        input = train_set[sample_idx][0].unsqueeze(1).to(model_device)
        
        with torch.no_grad():
            output = model(input).squeeze(1).cpu().numpy()

        # Plot original image
        ax_orig = axes[i, 0]
        ax_orig.imshow(input.squeeze().cpu().numpy(), cmap='gray')
        ax_orig.axis('off')
        ax_orig.set_title(f"Original {sample_idx}")
        
        # Plot predicted image
        ax_pred = axes[i, 1]
        ax_pred.imshow(output.squeeze(), cmap='gray')
        ax_pred.axis('off')
        ax_pred.set_title(f"Reconstructed {sample_idx}")

    with tempfile.TemporaryDirectory() as tmp:
        path = Path(tmp, "orig_predicted_plot.png")
        plt.savefig(path)
        mlflow.log_artifact(path)
    
    if show:
        plt.tight_layout()
        plt.show()

## Shape post-convolución

Shape:
* Input: $(C_{in}, H_{in}, W_{in})$
* Output: $(C_{out}, H_{out}, W_{out})$,

where:

\begin{align*}
H_{out} = \left\lfloor \frac{H_{in} + 2 \times \text{padding}[0] - \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) - 1}{\text{stride}[0]} + 1 \right\rfloor
\\
W_{out} = \left\lfloor \frac{W_{in} + 2 \times \text{padding}[1] - \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) - 1}{\text{stride}[1]} + 1 \right\rfloor
\end{align*}

In [ ]:
def post_conv_shape(
    h_in,
    w_in,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    dilation=1
):
    h_out = int(((h_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride) + 1)
    w_out = int(((w_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride) + 1)
    return out_channels, h_out, w_out

## Shape post-convolución transpuesta

Shape:
* Input: $(C_{in}, H_{in}, W_{in})$
* Output: $(C_{out}, H_{out}, W_{out})$,

where:

\begin{align*}
H_{out} = (H_{in} - 1) \times \text{stride}[0] - 2 \times \text{padding}[0] + \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) + \text{output\_padding}[0] + 1
\\
W_{out} = (W_{in} - 1) \times \text{stride}[1] - 2 \times \text{padding}[1] + \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) + \text{output\_padding}[1] + 1
\end{align*}

In [ ]:
def post_transconv_shape(
    h_in,
    w_in,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    output_padding=0,
    dilation=1
):
    h_out = (h_in - 1) * stride - 2 * padding + dilation * (kernel_size - 1) + output_padding + 1
    w_out = (w_in - 1) * stride - 2 * padding + dilation * (kernel_size - 1) + output_padding + 1
    return out_channels, h_out, w_out

## Shape post-max pooling

Shape:
* Input: $(C_{in}, H_{in}, W_{in})$
* Output: $(C_{out}, H_{out}, W_{out})$,

where:

\begin{align*}
H_{out} = \left\lfloor \frac{H_{in} + 2 \times \text{padding}[0] - \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) - 1}{\text{stride}[0]} + 1 \right\rfloor
\\
W_{out} = \left\lfloor \frac{W_{in} + 2 \times \text{padding}[1] - \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) - 1}{\text{stride}[1]} + 1 \right\rfloor
\end{align*}

In [ ]:
def post_maxpool_shape(
    h_in,
    w_in,
    out_channels,
    kernel_size,
    stride=None,
    padding=0,
    dilation=1
):
    if stride is None:
        stride = kernel_size
    h_out = int(((h_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride) + 1)
    w_out = int(((w_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride) + 1)
    return out_channels, h_out, w_out

# **1) Red Neuronal Autoencoder Convolucional de varias capas**
Defina y cree una red neuronal autoenconder convolucional. El autoencoder debe poseer dos módulos, un
encoder y un decoder. 
* El encoder tienen que tener al menos dos capas convolucionales 2D y una lineal.
* El decoder tiene que realizar una transformación aproximadamente inversa, por ejemplo, utilizando
primero una capa lineal y luego dos convolucionales traspuestas.
Recuerde incluir dropout, si es que lo considera necesario, y elegir adecuadamente el tipo de unidades de
activación de la capa de salida. No utilice dropout en la capa de salida. Por qué?

## Out channels: **1 -> 8 -> 16**

In [ ]:
class ConvAutoencoder2Layers_v1(nn.Module):
    def __init__(self, add_linear, p):
        super().__init__()
        self._add_linear = add_linear
        self._dropout = p
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3),    # (1,28,28) -> (8,26,26)
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.MaxPool2d(kernel_size=2),    # (8,26,26) -> (8,13,13)

            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),   # (8,13,13) -> (16,11,11)
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.MaxPool2d(kernel_size=2)    # (16,11,11) -> (16,5,5)
        )
        self.linear = nn.Sequential(
            nn.Flatten(),    # (16,5,5) -> 16x5x5
            nn.Linear(in_features=16*5*5, out_features=16*5*5),
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.Unflatten(dim=1, unflattened_size=(16,5,5)),
        )
        self.decoder = nn.Sequential(
            # nn.Unflatten(dim=1, unflattened_size=(16,5,5)),
            
            nn.ConvTranspose2d(
                in_channels=16, out_channels=8, kernel_size=3, stride=3, padding=1
            ),    # (16,5,5) -> (8,13,13)
            
            nn.ConvTranspose2d(
                in_channels=8, out_channels=1, kernel_size=4, stride=2, padding=0
            ),    # (8,13,13) -> (1,28,28)
            
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        if self._add_linear:
            x = self.linear(x)
        x = self.decoder(x)
        return x

In [ ]:
class ConvAutoencoder2Layers_v2(nn.Module):
    def __init__(self, add_linear, p):
        super().__init__()
        self._add_linear = add_linear
        self._dropout = p
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1),    # (1,28,28) -> (8,28,28)
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.MaxPool2d(kernel_size=2),    # (8,26,26) -> (8,14,14)

            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1),   # (8,14,14) -> (16,14,14)
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.MaxPool2d(kernel_size=2)    # (16,14,14) -> (16,7,7)
        )
        self.linear = nn.Sequential(
            nn.Flatten(),    # (16,7,7) -> 16*7*7
            nn.Linear(in_features=16*7*7, out_features=16*7*7),
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.Unflatten(dim=1, unflattened_size=(16,7,7)),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=16, out_channels=8, kernel_size=3, stride=2, padding=1, output_padding=1
            ),    # (16,7,7) -> (8,14,14)
            nn.ReLU(),
            nn.ConvTranspose2d(
                in_channels=8, out_channels=1, kernel_size=3, stride=2, padding=1, output_padding=1
            ),    # (8,14,14) -> (1,28,28)
            
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        if self._add_linear:
            x = self.linear(x)
        x = self.decoder(x)
        return x

## Out channels: **1 -> 8 -> 16 -> 32**

In [ ]:
class ConvAutoencoder3Layers_v1(nn.Module):
    def __init__(self, add_linear, p):
        super().__init__()
        self._add_linear = add_linear
        self._dropout = p
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3),    # (1,28,28) -> (8,26,26)
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.MaxPool2d(kernel_size=2),    # (8,26,26) -> (8,13,13)

            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),   # (8,13,13) -> (16,11,11)
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.MaxPool2d(kernel_size=2),    # (16,11,11) -> (16,5,5)
            
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),   # (16,5,5) -> (32,3,3)
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.MaxPool2d(kernel_size=2)    # (32,3,3) -> (32,1,1)
        )
        self.linear = nn.Sequential(
            nn.Flatten(),    # (32,1,1) -> 32*1*1
            nn.Linear(in_features=32*1*1, out_features=32*1*1),
            nn.ReLU(),
            nn.Dropout(p=p)
        )
        self.decoder = nn.Sequential(
            nn.Unflatten(dim=1, unflattened_size=(32, 1, 1)),
            
            nn.ConvTranspose2d(
                in_channels=32, out_channels=16, kernel_size=5
            ),    # (32,1,1) -> (16,5,5)
            
            nn.ConvTranspose2d(
                in_channels=16, out_channels=8, kernel_size=3, stride=3, padding=1
            ),    # (16,5,5) -> (8,13,13)
            
            nn.ConvTranspose2d(
                in_channels=8, out_channels=1, kernel_size=4, stride=2, padding=0
            ),    # (8,13,13) -> (1,28,28)
            
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        if self._add_linear:
            x = self.linear(x)
        x = self.decoder(x)
        return x

## Out channels: **1 -> 4 -> 8 -> 16**

In [ ]:
class ConvAutoencoder3Layers_v2(nn.Module):
    def __init__(self, add_linear, p):
        super().__init__()
        self._add_linear = add_linear
        self._dropout = p
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3),    # (1,28,28) -> (4,26,26)
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.MaxPool2d(kernel_size=2),    # (4,26,26) -> (4,13,13)

            nn.Conv2d(in_channels=4, out_channels=8, kernel_size=1),   # (4,13,13) -> (8,13,13)
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.MaxPool2d(kernel_size=2),    # (8,13,13) -> (8,6,6)
            
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),   # (8,6,6) -> (16,4,4)
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.MaxPool2d(kernel_size=2)    # (16,4,4) -> (16,2,2)
        )
        self.linear = nn.Sequential(
            nn.Flatten(),    # (16,2,2) -> 16*2*2
            nn.Linear(in_features=16*2*2, out_features=16*2*2),
            nn.ReLU(),
            nn.Dropout(p=p)
        )
        self.decoder = nn.Sequential(
            nn.Unflatten(dim=1, unflattened_size=(16, 2, 2)),
            
            nn.ConvTranspose2d(
                in_channels=16, out_channels=8, kernel_size=5
            ),    # (16, 2, 2) -> (8, 6, 6)
            
            nn.ConvTranspose2d(
                in_channels=8, out_channels=4, kernel_size=3, stride=2
            ),    # (8, 6, 6) -> (4, 13, 13)
            
            nn.ConvTranspose2d(
                in_channels=4, out_channels=1, kernel_size=4, stride=2
            ),    # (4, 13, 13) -> (1, 28, 28)
            
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        if self._add_linear:
            x = self.linear(x)
        x = self.decoder(x)
        return x

# **2) Entrenando el autoencoder**

1. Implemente en una función train_loop un loop que itere en modo entrenamiento sobre los batchs
o lotes de una época de entrenamiento.
2. Implemente en una función eval_loop un loop que itere en modo evaluación sobre los batchs de
una  ́epoca de entrenamiento.
3. Inicialize los DataLoaders sobre el conjunto de entranmiento y el conjunto de validación, usando
batchs de 100 ejemplos. AYUDA: Crear nuevas clases derivadas de la clase Dataset que sirvan para
entrenar autoencoders, i.e. en donde tanto el input como el output sean la misma imagen.
4. Cree una función de pérdida usando el Error Cuadrático Medio (ECM).
5. Cree un optimizador con un learning rate igual a 10−3. Pruebe con ADAM.
6. Cree una instancia del modelo autoencoder.
7. Especifique en que dispositivo (device) va a trabajar. Lo har ́a en una CPU, en una GPU o una
TPU?
8. Implemente un loop que itere sobre épocas de entrenamiento. Este loop debe guardar en listas
correspondientes, y en función de las épocas, los promedios del ECM sobre los conjuntos de entrenamiento y validación, inclueyendo los valores incorrectamente estimados del ECM sobre el conjunto de
entrenamiento. IMPORTANTE: No olvide copiar los batchs al dispositivo de trabajo.
9. Entrene y valide el modelo.
10. Grafique los distintos valores del ECM en función de las épocas de entrenamiento. Cual es el
número  ́optimo de  ́epocas de entrenamiento? Discuta y comente.
11. Grafique, comparativamente, algunas de las imagenes a predecir vs las imagenes predichas por el
modelo entrenado.
12. Repita utilizando variaciones de los hiperparámetros del autencoder. Por ejemplo, utilizando capas lineales de otros tamaños, valores distintos de dropout, otros optimizadores, distintos learning rates,
distintas transformaciones convolucionales y convolucionales traspuestas, etc. Que valores de estos hiperparámetros considera los más convenientes? Por qué?

In [ ]:
torch.cuda.empty_cache()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = 100
train_loader = DataLoader(
    CustomDataset(train_set_orig), batch_size=batch_size, shuffle=True, num_workers=4
)
valid_loader = DataLoader(
    CustomDataset(valid_set_orig), batch_size=batch_size, shuffle=False, num_workers=4
)
loss_fn = nn.MSELoss()

epochs = 50
lr = 10e-3
dropout = 0
add_linear = True

extra_hyperparms = {
    "dropout": dropout,
    "add_linear": add_linear
}

In [ ]:
model = ConvAutoencoder2Layers_v1(add_linear=add_linear, p=dropout).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

timestamp = time.strftime("%Y%m%d-%H%M%S")
with mlflow.start_run(run_name=f"run_{timestamp}"):
    results = train_validate_loop(
        model, train_loader, valid_loader, loss_fn, optimizer, epochs, extra_hyperparms
    )

    mlflow.pytorch.log_model(results['model'], "models")
    
    losses_plot(
        results['avg_losses_training'],
        results['train_avg_losses'],
        results['valid_avg_losses']
    )
    plot_orig_predicted(model, train_set_orig)

In [ ]:
model = ConvAutoencoder2Layers_v2(add_linear=add_linear, p=dropout).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

timestamp = time.strftime("%Y%m%d-%H%M%S")
with mlflow.start_run(run_name=f"run_{timestamp}"):
    results = train_validate_loop(
        model, train_loader, valid_loader, loss_fn, optimizer, epochs, extra_hyperparms
    )
    
    mlflow.pytorch.log_model(results['model'], "models")
    
    losses_plot(
        results['avg_losses_training'],
        results['train_avg_losses'],
        results['valid_avg_losses']
    )    
    plot_orig_predicted(model, train_set_orig)

# **3) Definiendo y entrenando un clasificador convolucional reutilizando el encoder**

1. Defina y cree un clasificador convolucional, agregando una capa clasificadora al encoder del
autoencoder previamente entrenado.
2. Reimplemente las funciones de entrenamiento, teniendo en cuenta que ahora debe incluir el cálculo
de precisiones.
3. Cree una función de pérdida usando la Cross Entropy Loss (CEL).
4. Cree una instancia del modelo clasificador.
5. Entrene y valide el modelo.
6. Grafique los distintos valores de la CEL y la precisión calculados, en función de las  ́epocas de
entrenamiento.
7. Utilice el conjunto de validación para calcular una Matriz de confusión. Grafíquela y comente los
resultados.

# **4) Prentrenamiento**
Modifique el optimizador para que solo reentrene los par ́ametros de la capa clasificadora, dejando los
par ́ametros de la capa codificadora tal como vienen entrenada del el autoencoder convolucional. Repita
los experimentos de la parte 3. Qué observa? Comente

Ayuda: Se recomienda guardar en archivos los pesos de las distintas capas de las redes entrenadas para
que puedan ser reutilizadas.